In [1]:
from torch import nn
import torch
import torchvision
import pandas as pd
import numpy as np
import PIL
import os
from torchinfo import summary
import matplotlib.pyplot as plt
import torch_directml

In [2]:
gpu=torch_directml.default_device()
gpu=torch_directml.device(0)

In [3]:
y_train=torch.Tensor(np.array((pd.get_dummies(pd.read_csv("archive/Training_set.csv")["label"])).replace({True:1,False:0}))).to(gpu)
y_train

C:\Users\praty\AppData\Local\Temp\ipykernel_18260\1776222190.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train=torch.Tensor(np.array((pd.get_dummies(pd.read_csv("archive/Training_set.csv")["label"])).replace({True:1,False:0}))).to(gpu)


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='privateuseone:0')

In [4]:
l=os.listdir("./archive/train/")
l=l[:10]

In [5]:
transformer=torchvision.transforms.Compose([torchvision.transforms.PILToTensor()])
for i in range(len(l)):
    img=PIL.Image.open(f"./archive/train/{l[i]}")
    tensor=transformer(img)
    l[i]=tensor
l=torch.Tensor(np.array(l)).to(gpu)
l.shape

torch.Size([10, 3, 224, 224])

In [6]:
class ImageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=2), # in=(3,224,224) out=(32,112,112)
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=16,kernel_size=2,padding=1), # in=(16,112,112) out=(16,56,56)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1,stride=1), # in=(16,56,56) out=(16,56,56)
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear((401408*2),512),
            nn.ReLU(),
            nn.Linear(512,75),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.layer1(x)

In [7]:
model=ImageModel().to(gpu)
summary(model)

Layer (type:depth-idx)                   Param #
ImageModel                               --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       416
│    └─ReLU: 2-2                         --
│    └─Conv2d: 2-3                       2,064
│    └─ReLU: 2-4                         --
│    └─MaxPool2d: 2-5                    --
│    └─ReLU: 2-6                         --
│    └─Flatten: 2-7                      --
│    └─Linear: 2-8                       411,042,304
│    └─ReLU: 2-9                         --
│    └─Linear: 2-10                      38,475
│    └─Sigmoid: 2-11                     --
Total params: 411,083,259
Trainable params: 411,083,259
Non-trainable params: 0

In [8]:
loss_func=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(lr=0.01,params=model.parameters())

In [9]:
batch1=l
print(batch1.shape)
batch1_y=y_train[:10].to(gpu)
batch1_y.shape

torch.Size([10, 3, 224, 224])


torch.Size([10, 75])

In [10]:
epochs=1
for epoch in range(epochs):
    model.train()
    y_preds=model.forward(batch1)
    print(y_preds)
    loss=loss_func(y_preds,batch1_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)

tensor([[3.5397e-02, 3.7720e-04, 7.4046e-01, 7.1188e-01, 3.1827e-01, 9.9365e-01,
         9.9453e-01, 2.3078e-03, 9.9803e-01, 8.4962e-01, 6.7458e-02, 4.9705e-01,
         1.4146e-01, 9.9412e-01, 1.8873e-01, 9.8385e-01, 8.0719e-01, 9.7146e-01,
         1.6358e-01, 6.4688e-03, 9.5346e-01, 9.4978e-01, 8.9031e-01, 8.2407e-02,
         9.9469e-01, 6.5240e-01, 2.3018e-02, 9.1637e-01, 2.6946e-01, 9.9573e-01,
         5.0686e-03, 1.3615e-02, 9.9831e-01, 4.5788e-01, 7.5330e-01, 9.9891e-01,
         5.9662e-01, 4.3075e-01, 9.9851e-01, 1.0538e-03, 8.9694e-01, 1.3569e-03,
         8.1040e-02, 5.1208e-01, 6.8113e-01, 7.7904e-01, 1.2408e-04, 9.5866e-01,
         9.2396e-01, 2.6740e-02, 4.3957e-01, 2.7608e-01, 8.1829e-01, 3.2992e-01,
         3.1828e-01, 1.1657e-01, 9.8409e-01, 5.5935e-02, 1.4783e-01, 2.1650e-01,
         4.5730e-01, 2.5691e-02, 2.1965e-02, 4.2917e-02, 9.1013e-01, 9.7713e-01,
         9.9875e-01, 7.4709e-03, 9.3321e-01, 6.7212e-01, 1.8582e-02, 7.2603e-04,
         3.5030e-01, 8.1998e

RuntimeError: Could not allocate tensor with 1644167168 bytes. There is not enough GPU video memory available!

In [12]:
x=1234

y_res=(model.forward(l[x:x+1]).cpu().detach().numpy())
print(y_res.argmax())

RuntimeError: The parameter is incorrect.

In [13]:
y_train[x:x+1].argmax()

tensor(58, device='privateuseone:0')